In [1]:
import pandas as pd
import os

In [2]:
dataset_path = '~/Workspace/github.com/amazon-science/esci-data/shopping_queries_dataset/'

# EDA

## examples

In [3]:
df_examples = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_examples.parquet'))

In [5]:
df_examples.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train


In [17]:
df_examples.groupby(['product_locale']).size()

product_locale
es     356410
jp     446053
us    1818825
dtype: int64

In [7]:
df_examples[df_examples['product_locale']=='jp'].head(10)

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
3502,3502,&ハニー シャンプー,116,B09GTY16XW,jp,S,1,1,train
3503,3503,&ハニー シャンプー,116,B07WTBVY3S,jp,E,1,1,train
3504,3504,&ハニー シャンプー,116,B07X4Y3WHX,jp,S,1,1,train
3505,3505,&ハニー シャンプー,116,B07X8GQTCX,jp,E,1,1,train
3506,3506,&ハニー シャンプー,116,B07XPBDWZ6,jp,S,1,1,train
3507,3507,&ハニー シャンプー,116,B07Y35KZY2,jp,S,1,1,train
3508,3508,&ハニー シャンプー,116,B07ZTGR26H,jp,E,1,1,train
3509,3509,&ハニー シャンプー,116,B086ZS4RT1,jp,S,1,1,train
3510,3510,&ハニー シャンプー,116,B07WS6TRZB,jp,E,1,1,train
3511,3511,&ハニー シャンプー,116,B08D34K35P,jp,S,1,1,train


In [14]:
df_examples.groupby(['small_version', 'large_version']).size()

small_version  large_version
0              1                1503277
1              1                1118011
dtype: int64

In [15]:
df_examples.groupby(['split']).size()

split
test      638016
train    1983272
dtype: int64

In [16]:
df_examples.groupby(['esci_label']).size()

esci_label
C      75652
E    1708158
I     263165
S     574313
dtype: int64

In [19]:
esci_label2gain = {
        'E' : 1.0,
        'S' : 0.1,
        'C' : 0.01,
        'I' : 0.0,
    }

## products

In [8]:
df_products = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_products.parquet'))

In [9]:
df_products.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es


In [10]:
df_products.shape

(1814924, 7)

## sources

In [21]:
df_sources = pd.read_csv(os.path.join(dataset_path, 'shopping_queries_dataset_sources.csv'))

In [24]:
df_sources.head()

,query_id,source
0,0,other
1,1,negations
2,2,negations
3,3,negations
4,4,behavioral


In [25]:
df_sources.groupby(['split']).size()

source
behavioral         3780
negations          6964
nlqec               169
other            113656
parse_pattern      6083
dtype: int64

# 実験に使うものだけのデータセットに絞る
## 条件
- product_locale='jp'のもの
- small_versionのもの
## 作成するデータセット
- 検索エンジンに入れる商品のデータセット(train, test)
- 精度を評価するためのクエリと商品についてのデータセット(train, test)

In [54]:
df_examples = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_examples.parquet'))

In [55]:
mask = (df_examples['product_locale']=='jp') & (df_examples['small_version']==1)
df_examples = df_examples[mask].copy()

In [56]:
df_examples.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
3502,3502,&ハニー シャンプー,116,B09GTY16XW,jp,S,1,1,train
3503,3503,&ハニー シャンプー,116,B07WTBVY3S,jp,E,1,1,train
3504,3504,&ハニー シャンプー,116,B07X4Y3WHX,jp,S,1,1,train
3505,3505,&ハニー シャンプー,116,B07X8GQTCX,jp,E,1,1,train
3506,3506,&ハニー シャンプー,116,B07XPBDWZ6,jp,S,1,1,train


In [57]:
df_examples.shape

(297883, 9)

In [58]:
df_examples_train = df_examples[df_examples['split']=='train']
df_examples_test = df_examples[df_examples['split']=='test']

In [59]:
df_products = pd.read_parquet(os.path.join(dataset_path, 'shopping_queries_dataset_products.parquet'))

In [61]:
mask = df_products['product_locale'] == 'jp'
df_products = df_products[mask].copy()

In [62]:
df_products.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
933783,B06WVHDWVR,(エイトトウキョウ)eight tokyo サングラス スタイリッシュ スクエア メンズ レ...,【おすすめＰＯＩＮＴ】<br> ■紫外線カット率99パーセント以上。あなたの目と肌をお守りし...,eight tokyoは株式会社エイトの商標登録アイウェアブランドです。紫外線（ＵＶ）カット...,eight tokyo(エイトトウキョウ),ブラック/ライトブルー,jp
933784,B06X17FJN7,What Say カラーレンズ ミラーレンズ サングラス クリアレンズ 伊達メガネ 全17色...,ソフト ＆ ハード ケース はランダムとなります。<br> また、弊社都合により 付属品 メ...,【 セット内容 】 ： オリジナルバンブーメガネケース・マイクロファイバークロスケース・商品...,What Say,ライト ブルー/ブラック,jp
933785,B071KNF11Z,アジアンサイズ 最適装着感 偏光レンズ 完全紫外線カット オリジナル スポーツサングラス PB15,<p>１、最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれ...,最適着用感：アジアンサイズよりフレーム形状を微調整、テンプルを長くまるく、かこまれた安定ホー...,SHD-PB,PB15,jp
933786,B0749LSC8N,Duduma 偏光 レンズ メンズスポーツサングラス 超軽量 UV400 紫外線をカット ／...,<b><br>商品紹介<br></b><br><b>1、Dudumaの特別なところのを発見<...,偏光レンズ:99%のUV400保護コーティング、有害な紫外線を100％ブロック、乱反射をカッ...,Duduma,ブラックマットフレーム/ブラックレンズ,jp
933787,B079NYT27C,PAERDE サングラス メンズ 偏光レンズ 運転 軽量 uvカット 紫外線カット 軽量 運...,None,None,PAERDE,None,jp


### trainとtestの関係を確認

In [63]:
len(set(df_examples_train['query_id'].unique()) & set(df_examples_test['query_id'].unique()))

0

In [64]:
len(set(df_examples_train['product_id'].unique()) & set(df_examples_test['product_id'].unique()))

17543

### 検索エンジンに入れる商品のデータセットを作成

In [66]:
train_product_ids = set(df_examples_train['product_id'].values)
test_product_ids = set(df_examples_test['product_id'].values)

In [67]:
# trainとtestで共通の商品があるとleakするので除外する
exclude_product_ids = train_product_ids & test_product_ids

In [70]:
df_products_train = df_products[df_products['product_id'].isin(train_product_ids-exclude_product_ids)]
df_products_test = df_products[df_products['product_id'].isin(test_product_ids-exclude_product_ids)]

In [71]:
df_products_train.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
933801,B00FW60P84,ゼロの焦点,None,None,None,None,jp
933802,B00MFBKQKG,第3話,None,None,None,None,jp
933803,B075QW7SBN,第08話「総理暗殺未遂事件」10年前の真実,None,None,None,None,jp
933804,B07H7WY84W,第67話 さらば！ 我が師よ我が友よ,None,None,None,None,jp
933805,B07HR6BC88,第06話,None,None,None,None,jp


In [72]:
df_products_train.shape

(153893, 7)

In [73]:
df_products_test.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
933884,B06XK834VC,Eyoyo 10インチ モニター TFTディスプレイ AV/VGA/TV/HDMI/ BNC...,仕様：<br> 画面サイズ：10.1インチ<br> 物理的な解像度：1366x768<br>...,【10インチモニター1366x768 IPSモニター】この小型HDMIモニターは、1366x...,Eyoyo,None,jp
933885,B073F63QF1,【178°全視野モニター 7インチ】1080P HDMI液晶モニター IPS 小型ディスプレ...,<b>説明：</b><br> Raspberry Pi 3B 2B/B/B+.A+ Aための...,【178°IPS 新モニター】:178°の広視野角、3辺が狭い、遮るもののない感覚；IPSデ...,Kuman,1,jp
933886,B0784R7GNB,WIMAXIT 11.6インチIPS モニター ポータブルモニター1920x1080 HDM...,ご注意：このモニターはHDCPには対応していません。著作権とプライバシーを保護するために、す...,【フルHD表示】WIMAXIT 11.6インチ大きな画面のIPSモニター、フルHD 1920...,WIMAXIT,None,jp
933887,B07JGF9RHY,10.1インチ 液晶 小型 モニター 1024X600解像度 車載ディスプレイ スピカー内蔵...,None,【高品質の解像度】：1024X600解像度で、高精細度の画質を再現する事が可能、明るく鮮やか...,OREK,None,jp
933888,B07PS89QGQ,モニター 10インチ ディスプレイ VGA/BNC/HDMIケーブル付き 内蔵スピーカー R...,None,【高解像度】HD画面解像度：1024（H）*800（V）ピクセル、コントラスト：350：1、...,METIS,None,jp


In [74]:
df_products_test.shape

(62414, 7)

In [75]:
df_products_train.to_csv('../../data/products_train.csv', index=False)
df_products_test.to_csv('../../data/products_test.csv', index=False)

### 精度を評価するためのクエリと商品についてのデータセット(train, test)

In [83]:
df_examples_products_train = pd.merge(
    df_examples_train,
    df_products_train,
    how='inner',
    on='product_id',
)

In [84]:
df_examples_products_test = pd.merge(
    df_examples_test,
    df_products_test,
    how='inner',
    on='product_id',
)

In [87]:
df_examples_products_train.head()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
0,3502,&ハニー シャンプー,116,B09GTY16XW,jp,S,1,1,train,HONEYPLUS ハニープラス スムース&リペアシャンプー 900mL 詰め替え,🐝もこもこの弾力泡で傷んだ髪を優しく洗う、シリコーンフリーのアミノ酸系シャンプー。 🐝全成分...,🐝アミノ酸系シャンプーにハチミツを高配合。髪と地肌に優しい弾力泡が汚れをしっかり落とします。...,ハニープラス,None,jp
1,3503,&ハニー シャンプー,116,B07WTBVY3S,jp,E,1,1,train,ハニープラス エアリートリートメント 500ML ２個セット,ふんわりまとまる!髪と地肌にやさしい生ハチミツを使った柔らかい髪用トリートメント!サロン品質...,ハニープラス エアリートリートメント 500ML ２個セット,ハニープラス,None,jp
2,3504,&ハニー シャンプー,116,B07X4Y3WHX,jp,S,1,1,train,アンドハニー ディープモイスト シャンプー トリートメント 詰め替え セット『超しっとり オ...,【商品説明】 特徴（特長） ●90％以上配合された保湿&保護成分が、髪に潤いを閉じ込めます。...,None,＆ｈｏｎｅｙ,ディープモイスト,jp
3,3505,&ハニー シャンプー,116,B07X8GQTCX,jp,E,1,1,train,Dove(ダヴ) ダヴ ボタニカルセレクション ナチュラルシャイン シャンプー つめかえ用 ...,None,サイズ:350g\n原産国:日本\n内容量:350g\n髪質:ノーマル,Dove(ダヴ),None,jp
4,704590,dove botanical selection,35060,B07X8GQTCX,jp,S,1,1,train,Dove(ダヴ) ダヴ ボタニカルセレクション ナチュラルシャイン シャンプー つめかえ用 ...,None,サイズ:350g\n原産国:日本\n内容量:350g\n髪質:ノーマル,Dove(ダヴ),None,jp


In [86]:
df_examples_products_test.head()

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
0,12756,0.4 ノズル,438,B08PBJSJM8,jp,S,1,1,test,エアブラシ ダブルアクションエアーブラシ セットUSB充電式エアブラシ 重力式 2種類ノズル...,初心者でも簡単: 電源を入れて、レバーを下に押したまま手前に引くだけで、塗料がシューッと出ま...,高性能ダブルアクションエアブラシ:噴霧効果に優れた充電式エアブラシなので、塗料を細かい霧状に...,Hotorda,None,jp
1,12757,0.4 ノズル,438,B087R9X5X7,jp,E,1,1,test,GANZTON 19B エアブラシ セット 0.4mmノズル ミニコンプレッサー クリーナー...,<b>製品仕様：</b><br> サイズ：120 * 70 * 90mm<br> 色：白、赤...,【エアブラシコンプレッサーセット】エアブラシアートを始めるために必要な、エアブラシ付きのミニ...,GANZTON,クリーニングキット付ホワイト,jp
2,12758,0.4 ノズル,438,B0881K8XHS,jp,S,1,1,test,Creality ホットエンド ブロックシリコーン 保護カバー 高温耐性 3個 + 0.4m...,仕様：<br> 材質：耐熱性、非粘着性シリコーン<br> <br> サイズ：23 x 23 ...,280℃に耐えるシリコンカバーで、優れた耐熱性を持ちます。ほとんどの3Dプリンターに付属した...,Creality 3D,ブラック,jp
3,12759,0.4 ノズル,438,B089VZ3RY8,jp,E,1,1,test,TRONXY 3Dアップグレードパーツ組み立て済みMK10エクストルーダーホットエンドキット...,null<b>特徴：</b><br /> MK10で組み立てられたエクストルーダーホットエン...,MK10で組み立てられたエクストルーダーホットエンドキットは、X5SA / X5SAPRO ...,Aibecy1,None,jp
4,12760,0.4 ノズル,438,B08H142Z8H,jp,S,1,1,test,FULARR® 3m PTFEテフロンチューブ、6個 PC4-M6フィッティング、6個 PC...,<b>仕様：<br> タイプ：PTFEテフロンチューブ <br> ●材質：PTFE(ポリテト...,✪ パッケージ内容：3m PTFTテフロンチューブ、6個PC4-M6フィッティング、6個PC...,FULARR,None,jp


In [93]:
esci_label2gain = {
        'E' : 1.0,
        'S' : 0.1,
        'C' : 0.01,
        'I' : 0.0,
}

In [95]:
df_examples_products_train['esci_label'] = df_examples_products_train['esci_label'].map(esci_label2gain)
df_examples_products_test['esci_label'] = df_examples_products_test['esci_label'].map(esci_label2gain)

In [96]:
use_cols = ['example_id', 'query', 'query_id', 'product_id','esci_label', 'product_title',
            'product_description', 'product_bullet_point', 'product_brand', 'product_color']

In [98]:
df_examples_products_train[use_cols].head()

,example_id,query,query_id,product_id,esci_label,product_title,product_description,product_bullet_point,product_brand,product_color
0,3502,&ハニー シャンプー,116,B09GTY16XW,0.1,HONEYPLUS ハニープラス スムース&リペアシャンプー 900mL 詰め替え,🐝もこもこの弾力泡で傷んだ髪を優しく洗う、シリコーンフリーのアミノ酸系シャンプー。 🐝全成分...,🐝アミノ酸系シャンプーにハチミツを高配合。髪と地肌に優しい弾力泡が汚れをしっかり落とします。...,ハニープラス,None
1,3503,&ハニー シャンプー,116,B07WTBVY3S,1.0,ハニープラス エアリートリートメント 500ML ２個セット,ふんわりまとまる!髪と地肌にやさしい生ハチミツを使った柔らかい髪用トリートメント!サロン品質...,ハニープラス エアリートリートメント 500ML ２個セット,ハニープラス,None
2,3504,&ハニー シャンプー,116,B07X4Y3WHX,0.1,アンドハニー ディープモイスト シャンプー トリートメント 詰め替え セット『超しっとり オ...,【商品説明】 特徴（特長） ●90％以上配合された保湿&保護成分が、髪に潤いを閉じ込めます。...,None,＆ｈｏｎｅｙ,ディープモイスト
3,3505,&ハニー シャンプー,116,B07X8GQTCX,1.0,Dove(ダヴ) ダヴ ボタニカルセレクション ナチュラルシャイン シャンプー つめかえ用 ...,None,サイズ:350g\n原産国:日本\n内容量:350g\n髪質:ノーマル,Dove(ダヴ),None
4,704590,dove botanical selection,35060,B07X8GQTCX,0.1,Dove(ダヴ) ダヴ ボタニカルセレクション ナチュラルシャイン シャンプー つめかえ用 ...,None,サイズ:350g\n原産国:日本\n内容量:350g\n髪質:ノーマル,Dove(ダヴ),None


In [97]:
df_examples_products_train[use_cols].to_csv('../../data/examples_train.csv', index=False)
df_examples_products_test[use_cols].to_csv('../../data/examples_test.csv', index=False)